In [1]:
from tricorder.swan import SWAN
from tricorder.cohort import ProcedureCohort
from tricorder.procedure_codesets import cabg_names
import pyarrow.csv as csv
import pyarrow.compute as pc
import pyarrow.plasma as plasma
import pyarrow as pa
import pandas as pd

In [2]:
swan = SWAN(root_dir='/Users/elijahc/data/compass/SWAN_20210210/')

In [3]:
teg_names = [
    'TEG: MAXIMUM AMPLITUDE', 
    'TEG: COAGULATION TIME', 
    'TEG: REACTION TIME', 
    'TEG: ALPHA ANGLE', 
    'TEG: LYSIS TIME 30 MINUTES',
    'R TIME',
    'K TIME',
]

procedures = [
    'TRICUSPID VALVE REPLACEMENT/REPAIR',
    'PULMONARY VALVE REPLACEMENT/REPAIR',
    'MITRAL VALVE REPLACEMENT/REPAIR',
    'CORONARY ARTERY BYPASS GRAFT TIMES 1-5 (CABG)',
    'AORTIC VALVE REPLACEMENT/REPAIR', 
]

In [4]:
client = plasma.connect('/tmp/plasma')

In [5]:
client.disconnect()

In [6]:
cam = swan.sel(procedures=cabg_names,flowsheet=['CAM ICU'])
cam.encounter_id.unique()

array([118100133870,  74960175079,  44894186181,  52627514512,
        62987849287,  73575338125,  76182778565,  49093837852,
        60128677923,  43822573422,  72128817774,  79473994810,
        63095777908,  38566043277,  78530522256,  76207316555,
       105945925798,  91439676358,  43934317224,  57369214164,
        56184035780,  68025969439,  58379625177,  70794218675,
       120951274342,  52993745632,  58645879981,  65382937549,
        58002045692,  69069204524,  52258222799,  86792681060,
        38411980126,  55708858658,  73704856526, 117269973440,
        70347960874,  59679259817,  64163918558,  68378395774,
        65506689663, 130725581359,  49252483444,  64196950499,
        74407504588, 106792658391,  99148241085,  75744740090,
        61114928662,  39012370369,  51381706449,  37804886489,
        95594825196,  90254530084,  77880686769,  39820406133,
       108018738212, 121176953034,  73225697759,  39996631976,
        72046212993, 103863913010,  89740383198,  61440

In [7]:
p = ProcedureCohort(db=swan,procedures=cabg_names, encounter_id=cam.encounter_id.unique())
p.gender

,encounter_id,gender
0,36893446239,Female
1,37053606074,Male
2,37153354239,Male
3,37227500558,Female
4,37352648614,Female
...,...,...
587,132730493704,Male
588,133188976982,Male
589,133265300024,Female
590,133885028964,Male


In [8]:
p.osa

,encounter_id,OSA
0,78595499467,True
1,96269546813,False
2,80111228795,False
3,72009540787,True
4,96228600591,True
...,...,...
586,133885028964,False
587,98538391054,False
589,87737667011,False
590,89740383198,False


In [21]:
p.db.flowsheet.sel(encounter_id=p.eid,display_name=['CCO','SPO2'],pivot=True)

display_name                                  CCO       SPO2
encounter_id flowsheet_days_since_birth                     
36893446239  19671                       3.715385  96.444444
             19672                       3.633333  97.916667
37053606074  28134                       4.221739  95.423077
             28135                       3.683784  96.729730
             28136                       3.652083  94.745455
...                                           ...        ...
133885028964 20039                       6.360000  95.057143
             20040                       5.950000  94.200000
143074677125 26357                       5.322222  98.809524
             26358                       6.233333  98.285714
             26359                       4.720000  99.500000

[1673 rows x 2 columns]

In [81]:
dat = p.db.sel(procedures=cabg_names,encounter_id=p.eid,flowsheet=['CCO','SPO2'])
hgb = p.db.sel(procedures=cabg_names,encounter_id=p.eid,labs=['HEMOGLOBIN','HEMOGLOBIN ARTERIAL','HEMOGLOBIN VENOUS'])
hgb['name'] = 'HEMOGLOBIN'
dat = pd.concat([dat,hgb])
dat.value = pd.to_numeric(dat.value,errors='coerce')
dat['time'] = pd.to_timedelta(dat.time) + pd.to_timedelta(dat.day,unit='days')

In [83]:
dat.groupby(['encounter_id','day','name']).count()

person_id  procedure  days_from_dob_procstart  \
encounter_id day name                                                        
36893446239  0.0 HEMOGLOBIN          5          5                        5   
             1.0 CCO                13         13                       13   
                 SPO2               18         18                       18   
             2.0 CCO                12         12                       12   
                 SPO2               12         12                       12   
...                                ...        ...                      ...   
143074677125 0.0 SPO2               21         21                       21   
             1.0 CCO                27         27                       27   
                 SPO2               28         28                       28   
             2.0 CCO                10         10                       10   
                 SPO2               10         10                       10   

                             days_from_dob  time  value  
encounter_id day name                                    
36893446239  0.0 HEMOGLOBIN              5     5      5  
             1.0 CCO                    13    13     13  
                 SPO2                   18    18     18  
             2.0 CCO                    12    12     12  
                 SPO2                   12    12     12  
...                                    ...   ...    ...  
143074677125 0.0 SPO2                   21    21     21  
             1.0 CCO                    27    27     27  
                 SPO2                   28    28     28  
             2.0 CCO                    10    10     10  
                 SPO2                   10    10     10  

[3896 rows x 6 columns]

In [86]:
dat.query('name == "HEMOGLOBIN"').groupby('encounter_id').count()

,person_id,procedure,days_from_dob_procstart,days_from_dob,day,time,name,value
encounter_id,,,,,,,,
36893446239,5,5,5,5,5,5,5,5
37053606074,7,7,7,7,7,7,7,7
37153354239,13,13,13,13,13,13,13,13
37227500558,5,5,5,5,5,5,5,5
37352648614,6,6,6,6,6,6,6,6
...,...,...,...,...,...,...,...,...
132730493704,13,13,13,13,13,13,13,13
133188976982,7,7,7,7,7,7,7,7
133265300024,9,9,9,9,9,9,9,9


In [87]:
dat.query('name == "HEMOGLOBIN"')[['encounter_id','day','time','name','value']].pivot_table(
    index=['encounter_id','day','time'],
    columns='name',
    values='value'
)

name                              HEMOGLOBIN
encounter_id day time                       
36893446239  0.0 0 days 11:55:00       11.70
                 0 days 14:15:00        8.20
                 0 days 14:48:00        8.20
                 0 days 15:10:00        7.70
                 0 days 15:34:00        7.40
...                                      ...
143074677125 0.0 0 days 09:38:00        8.70
                 0 days 11:52:00        8.10
                 0 days 10:07:00        7.15
                 0 days 10:49:00        6.80
                 0 days 07:49:00        9.40

[4895 rows x 1 columns]

In [88]:
import seaborn as sns

ModuleNotFoundError: No module named 'seaborn'

In [11]:
p.mechanical_ventilation_duration

,encounter_id,VENT DUR
0,36893446239,NaN
1,37053606074,0.0
2,37153354239,NaN
3,37227500558,NaN
4,37352648614,NaN
...,...,...
561,132730493704,NaN
562,133188976982,NaN
563,133265300024,NaN
564,133885028964,NaN


In [12]:
with swan.cohort(procedures=procedures) as c:
    icu_days = c.post_op_icu_days().count().reset_index()

In [13]:
# swan.labs.partition()
# swan.procedures.partition(column='person_id')
# swan.flowsheet.partition()
# swan.diagnosis.partition()

In [14]:
swan.diagnosis.unique()

array(['SYMPTOMATIC OLD MI (MYOCARDIAL INFARCTION)',
       'UNRESOLVED LOBAR PNEUMONIA (HC CODE)', 'SENIUM',
       'ESSENTIAL (PRIMARY) HYPERTENSION',
       'ENDOCARDITIS, VALVE UNSPECIFIED',
       'CARDIOMYOPATHY IN DISEASES CLASSIFIED ELSEWHERE',
       'PNEUMONIA DUE TO STREPTOCOCCUS PNEUMONIAE',
       'PNEUMONIA DUE TO HEMOPHILUS INFLUENZAE',
       'AGE-RELATED PHYSICAL DEBILITY',
       'HEMORRHAGE, NOT ELSEWHERE CLASSIFIED',
       'ENCOUNTER FOR FOLLOW-UP EXAMINATION AFTER COMPLETED TREATMENT FOR CONDITIONS OTHER THAN MALIGNANT NEOPLASM',
       'YOLK SAC TUMOUR', 'VENTRICULAR MYXOMA', 'USERS, COCAINE',
       'WITHDRAWAL FROM NICOTINE', 'RHEUMATIC MITRAL VALVE PROLAPSE',
       'RHEUMATIC MITRAL STENOSIS WITH REGURGITATION',
       'RHEUMATIC MITRAL VALVE FAILURE',
       'RHEUMATIC LEAKING, NARROWED AORTIC VALVE',
       'RHEUMATIC MITRAL AND AORTIC VALVE REGURGITATION',
       'STENOSIS AND INSUFFICIENCY OF MITRAL OR AORTIC VALVE WITH STENOSIS OR INSUFFICIENCY, OR BOTH,

In [15]:
swan.procedures.sel(order_name=procedures)

,encounter_id,order_name,days_from_dob_procstart,person_id
0,113856305537,MITRAL VALVE REPLACEMENT/REPAIR,21656,1001662573
1,78595499467,AORTIC VALVE REPLACEMENT/REPAIR,24922,1059873860
2,78595499467,CORONARY ARTERY BYPASS GRAFT TIMES 1-5 (CABG),24922,1059873860
3,96269546813,CORONARY ARTERY BYPASS GRAFT TIMES 1-5 (CABG),22751,1061539357
4,162370632423,MITRAL VALVE REPLACEMENT/REPAIR,24158,1118835508
...,...,...,...,...
1778,72691375624,MITRAL VALVE REPLACEMENT/REPAIR,12598,914707726
1779,81004618129,MITRAL VALVE REPLACEMENT/REPAIR,23988,928313935
1780,137603444009,CORONARY ARTERY BYPASS GRAFT TIMES 1-5 (CABG),24713,94890955
1781,70389777448,CORONARY ARTERY BYPASS GRAFT TIMES 1-5 (CABG),20783,987981159


In [16]:
teg = swan.sel(procedures=procedures, labs=teg_names)
teg

,person_id,encounter_id,procedure,days_from_dob_procstart,days_from_dob,day,time,name,value
0,7087724467,129946253665,CORONARY ARTERY BYPASS GRAFT TIMES 1-5 (CABG),26525,26525,0.0,12:15:00,K TIME,2.8
1,7087724467,129946253665,CORONARY ARTERY BYPASS GRAFT TIMES 1-5 (CABG),26525,26525,0.0,12:15:00,K TIME,2.5
2,7087724467,129946253665,CORONARY ARTERY BYPASS GRAFT TIMES 1-5 (CABG),26525,26525,0.0,12:15:00,R TIME,5.9
3,7087724467,129946253665,CORONARY ARTERY BYPASS GRAFT TIMES 1-5 (CABG),26525,26525,0.0,12:15:00,R TIME,5.7
4,7087724467,129946253665,CORONARY ARTERY BYPASS GRAFT TIMES 1-5 (CABG),26525,26525,0.0,12:15:00,TEG: ALPHA ANGLE,58.0
...,...,...,...,...,...,...,...,...,...
6305,3538737230,60528251099,CORONARY ARTERY BYPASS GRAFT TIMES 1-5 (CABG),22533,22533,0.0,17:35:00,TEG: REACTION TIME,4.8
6306,3548971039,60895836755,CORONARY ARTERY BYPASS GRAFT TIMES 1-5 (CABG),20919,20919,0.0,11:14:00,TEG: COAGULATION TIME,1.2
6307,3548971039,60895836755,CORONARY ARTERY BYPASS GRAFT TIMES 1-5 (CABG),20919,20919,0.0,11:14:00,TEG: LYSIS TIME 30 MINUTES,1.1
6308,3548971039,60895836755,CORONARY ARTERY BYPASS GRAFT TIMES 1-5 (CABG),20919,20919,0.0,11:14:00,TEG: MAXIMUM AMPLITUDE,69.3


In [17]:
len(proc_df.encounter_id.unique())

NameError: name 'proc_df' is not defined

In [ ]:
# tegs = csv.read_csv(swan.labs.file_path).to_pandas()
tegs = swan.labs.sel(lab_component_name=teg_names,encounter_id=proc_df.encounter_id.unique())
# tegs = tegs[tegs.encounter_id.isin(proc_df.encounter_id.unique())]
tegs
tegs.groupby('lab_component_name').count().sort_values(by='loinc_code',ascending=False)

In [ ]:
tab= csv.read_csv(swan.labs.file_path)

In [ ]:
tab

In [ ]:
ftab = tab.filter(pc.is_in(tab['lab_component_name'],options=pc.SetLookupOptions(value_set=pa.array(teg_names))))

In [ ]:
tab['lab_component_name'].to_pandas().unique()

In [ ]:
ftab['lab_component_name']

In [ ]:
tegs.groupby(['lab_component_name','encounter_id',]).count().sort_values('loinc_code',ascending=False).head(50)

In [ ]:
labs = swan.labs.sel(lab_component_name=teg_names, cache=False, encounter_id=proc_df.encounter_id.unique())

In [ ]:
len(labs.encounter_id.unique())

In [ ]:
len(proc_df.encounter_id.unique())

In [ ]:
import pyarrow.csv as csv

In [ ]:
labs = csv.read_csv(swan.labs.file_path).to_pandas()
labs = labs[labs.lab_component_name.isin(teg_names)]

In [ ]:
len(labs.encounter_id.unique())

In [ ]:
trans = csv.read_csv('/Users/elijahc/data/compass/SWAN_20210210/raw/Table5_Blood_Transfusion.csv').to_pandas()
trans.transfusion_name.unique()

In [ ]:
meds = csv.read_csv('/Users/elijahc/data/compass/SWAN_20210210/raw/Table4_Administered_Medication.csv').to_pandas()
meds

In [ ]:
labs = swan.labs.sel(lab_component_name=teg_names)
len(labs)

In [ ]:
len(labs.encounter_id.unique())